In [1]:
import requests
import pandas as pd
import numpy as np

In [3]:
URL="https://dogapi.dog/api/v2/breeds"

res=requests.get(URL)

In [4]:
json_data=res.json()

df=pd.json_normalize(json_data["data"])
df=df[["id","attributes.name","attributes.life.max"]]
df=df.rename(columns={"attributes.name":"breed_name","attributes.life.max":"max_life"})
df
df.groupby("max_life").count()

,id,breed_name
max_life,,
8,2,2
10,14,14
11,1,1
12,19,19
13,3,3
14,110,110
15,103,103
16,28,28
17,1,1


In [8]:
res=requests.get("https://www.scrapethissite.com/pages/simple/")

200


In [15]:
if(res.status_code==200):
    # print(res.text) #doc type html of page
    # print(res.content)
    print(res.headers)

# with open("scraped_data/data1.html","w",encoding="utf-8") as f:
#     f.write(res.text)

{'Date': 'Wed, 31 Dec 2025 10:31:01 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Nel': '{"report_to":"heroku-nel","response_headers":["Via"],"max_age":3600,"success_fraction":0.01,"failure_fraction":0.1}', 'Report-To': '{"group":"heroku-nel","endpoints":[{"url":"https://nel.heroku.com/reports?s=a6D%2FAUhEzSiq%2FcZr0pXALHHeTUh%2FvkUAWVD6tLw2MWM%3D\\u0026sid=67ff5de4-ad2b-4112-9289-cf96be89efed\\u0026ts=1767177061"}],"max_age":3600}', 'Reporting-Endpoints': 'heroku-nel="https://nel.heroku.com/reports?s=a6D%2FAUhEzSiq%2FcZr0pXALHHeTUh%2FvkUAWVD6tLw2MWM%3D&sid=67ff5de4-ad2b-4112-9289-cf96be89efed&ts=1767177061"', 'Server': 'cloudflare', 'Via': '1.1 heroku-router', 'cf-cache-status': 'DYNAMIC', 'Content-Encoding': 'zstd', 'CF-RAY': '9b691ada882bf4c0-MRS', 'alt-svc': 'h3=":443"; ma=86400'}


In [18]:
from bs4 import BeautifulSoup

with open("scraped_data/data1.html","r") as f:
    html_content=f.read()
soup=BeautifulSoup(html_content,"lxml")

In [41]:
all_h3=soup.find_all("h3")
all_countries=[]
for h3 in all_h3:
    name=h3.get_text(strip=True)#get_text fnc is used to access the content in h3 tag
    # population=h3.find_next("div").select("span.country-population")[0].get_text()
    population=h3.find_next("div").select_one("span.country-population").get_text(strip=True)

    all_countries.append([name,population])

In [50]:
import pandas as pd
df=pd.DataFrame(all_countries,columns=["Name","Population"])
df

,Name,Population
0,Andorra,84000
1,United Arab Emirates,4975593
2,Afghanistan,29121286
3,Antigua and Barbuda,86754
4,Anguilla,13254
...,...,...
245,Yemen,23495361
246,Mayotte,159042
247,South Africa,49000000
248,Zambia,13460305


In [51]:
df.to_csv("cleaned_data/data.csv",index=False) #list ka uper hum log to_csv nahi lagana hai only apply to data frames

In [130]:
# home work
res=requests.get("https://quotes.toscrape.com/")
if res.status_code==200:
    with open("scraped_data/quotes.html","w",encoding="utf8") as f:
        f.write(res.text)


In [131]:
with open("scraped_data/quotes.html","r",encoding="utf8") as f:
    html_content=f.read()
soup=BeautifulSoup(html_content)
all_span=soup.select("span.text")
final_data=[]
for span in all_span:
    all_tags=span.find_next("div").select("a.tag")
    for tag in all_tags:
        if tag.get_text()=="life":
            quote=span.get_text()
            author=span.find_next("span").select("small.author")[0].get_text()
            final_data.append([author,quote])


df=pd.DataFrame(final_data,columns=["Author","Quotes"])
df

,Author,Quotes
0,Albert Einstein,“There are only two ways to live your life. On...
1,André Gide,“It is better to be hated for what you are tha...


In [134]:
page_count=1
while True:
    URL=f"https://quotes.toscrape.com/page/{page_count}/"
    res=requests.get(URL)
    
    soup=BeautifulSoup(res.text,"lxml")
    quotes=soup.select("div.quote")
    
    if not quotes:
        print("no more pages available..")
        break

    with open(f"scraped_data/quotes{page_count}.html","w",encoding="utf8") as f:
        f.write(res.text)
        print(f"downloaded data from page{page_count}")

    page_count+=1
    

downloaded data from page1
downloaded data from page2
downloaded data from page3
downloaded data from page4
downloaded data from page5
downloaded data from page6
downloaded data from page7
downloaded data from page8
downloaded data from page9
downloaded data from page10
no more pages available..


In [154]:
page_count=11
life_quotes=[]
while page_count>=2:
    with open(f"scraped_data/quotes{page_count-1}.html","r",encoding="utf8") as f:
        html_content=f.read()
    
        soup=BeautifulSoup(html_content,"lxml")
        all_quotes=soup.select("div.quote")
        
        for q in all_quotes:
            all_tags=[]
            
            for tag in q.select(".tags .tag"):
                all_tags.append(tag.get_text())
    
            if "life" in all_tags:
                text=q.select_one("span.text").get_text()
                author=q.select_one("small.author").get_text()
                life_quotes.append([author,text])
    page_count-=1
df=pd.DataFrame(life_quotes,columns=["author","text"])
df

,author,text
0,Jimi Hendrix,“I'm the one that's got to die when it's time ...
1,Khaled Hosseini,“But better to get hurt by the truth than comf...
2,Mark Twain,“The fear of death follows from the fear of li...
3,Ralph Waldo Emerson,“Finish each day and be done with it. You have...
4,George Bernard Shaw,“Life isn't about finding yourself. Life is ab...
5,Albert Einstein,“Life is like riding a bicycle. To keep your b...
6,Dr. Seuss,"“Today you are You, that is truer than true. T..."
7,Marilyn Monroe,“This life is what you make it. No matter what...
8,Douglas Adams,"“I may not have gone where I intended to go, b..."
9,Mark Twain,"“Good friends, good books, and a sleepy consci..."
